<a href="https://colab.research.google.com/github/tmoura/ED/blob/master/Atividade4-Ciclo2-RESPOSTAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deslib

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from deslib.dcs import OLA
from deslib.dcs import LCA
from deslib.dcs import MCB
from deslib.des import KNORAE
from deslib.des import KNORAU

from sklearn.ensemble import RandomForestClassifier

url = "https://raw.githubusercontent.com/tmoura/machinelearning/master/datasets/ionosphere.data"

# Carregar base de dados
# DataFrame
dataset = pd.read_csv(url, header=None)

columns = len(dataset.columns)

y = dataset[0] # extrai a primeira coluna, que é o label
X = dataset.loc[:,1:columns-1]

# Transforma para Array NumPy
X = np.array(X)
y = np.array(y)

In [33]:
from sklearn.model_selection import StratifiedKFold

folds = 10

kf = StratifiedKFold(n_splits = folds)

## 10 conjuntos de dados
X_train = []
y_train = []

X_test = []
y_test = []

for train_index, test_index in kf.split(X,y):
  X_train.append(X[train_index])
  X_test.append(X[test_index])

  y_train.append(y[train_index])
  y_test.append(y[test_index])

In [34]:
resultsDC = []
resultsRF = []

for i in range(folds):
  dc = DecisionTreeClassifier(criterion="entropy")
  dc.fit(X_train[i], y_train[i])

  rf = RandomForestClassifier()
  rf.fit(X_train[i], y_train[i])

  resultsDC.append(dc.score(X_test[i], y_test[i]))
  resultsRF.append(rf.score(X_test[i], y_test[i]))


print('Árvore de Decisão: ', np.mean(resultsDC))
print('Random Forest: ', np.mean(resultsRF))

Árvore de Decisão:  0.8746825396825397
Random Forest:  0.9088095238095237


In [ ]:
from sklearn.ensemble import BaggingClassifier

resultsBAG = []
resultsKNE = []
resultsKNU = []
resultsLCA = []
resultsOLA = []
resultsMCB = []

for i in range(folds):
  bag = BaggingClassifier(n_estimators=50)
  bag.fit(X_train[i], y_train[i])

  resultsBAG.append(bag.score(X_test[i], y_test[i]))

  X_new, X_dsel, y_new, y_dsel = train_test_split(X_train[i], y_train[i], test_size=0.5,random_state=None)

  kne = KNORAE(bag, k=5)
  mcb = MCB(bag, k=5)
  lca = LCA(bag, k=5)
  ola = OLA(bag, k=5)
  knu = KNORAU(bag, k=5)

  # Fitting the des techniques
  kne.fit(X_dsel, y_dsel)
  mcb.fit(X_dsel, y_dsel)
  lca.fit(X_dsel, y_dsel)
  ola.fit(X_dsel, y_dsel)
  knu.fit(X_dsel, y_dsel)

  resultsKNE.append(kne.score(X_test[i], y_test[i]))
  resultsKNU.append(knu.score(X_test[i], y_test[i]))
  resultsLCA.append(lca.score(X_test[i], y_test[i]))
  resultsOLA.append(ola.score(X_test[i], y_test[i]))
  resultsMCB.append(mcb.score(X_test[i], y_test[i]))

# Calculate classification accuracy of each technique
print('KNORA-Eliminate: ',np.mean(resultsKNE))
print('KNORA-Union: ',np.mean(resultsKNU))
print('MCB: ',np.mean(resultsMCB))
print('OLA: ',np.mean(resultsOLA))
print('LCA: ',np.mean(resultsLCA))
print('Bagging: ',np.mean(resultsBAG))